# Customer Segmentation in Python

### Assign daily acquisition cohort
As you have seen in the video, defining a cohort is the first step to cohort analysis. You will now create daily cohorts based on the day each customer has made their first transaction.

The data has been loaded as online DataFrame, you can now print its header with online.head() in the console.
```python
# Define a function that will parse the date
def get_day(x): return dt.datetime(x.year, x.month, x.day) 

# Create InvoiceDay column
online['InvoiceDay'] = online['InvoiceDate'].apply(get_day) 

# Group by CustomerID and select the InvoiceDay value
grouping = online.groupby('CustomerID')['InvoiceDate'] 

# Assign a minimum InvoiceDay value to the dataset
online['CohortDay'] = grouping.transform('min')

# View the top 5 rows
print(online.head())

```
Calculate time offset in days - part 1
Calculating time offset for each transaction allows you to report the metrics for each cohort in a comparable fashion.

First, we will create 6 variables that capture the integer value of years, months and days for Invoice and Cohort Date using the get_date_int() function that's been already defined for you:
```python
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day
```
The online data has been loaded, you can print its header to the console by calling online.head().

```python
# Get the integers for date parts from the `InvoiceDay` column
invoice_year, invoice_month, invoice_day = get_date_int(online, 'InvoiceDay')

# Get the integers for date parts from the `CohortDay` column
cohort_year, cohort_month, cohort_day = get_date_int(online, 'CohortDay')

```
### Calculate time offset in days - part 2
Great work! Now, we have six different data sets with year, month and day values for Invoice and Cohort dates - invoice_year, cohort_year, invoice_month, cohort_month, invoice_day, and cohort_day.

In this exercise you will calculate the difference between the Invoice and Cohort dates in years, months and days separately and then calculate the total days difference between the two. This will be your days offset which we will use in the next exercise to visualize the customer count. The online data has been loaded, you can print its header to the console by calling online.head().



```python
# Calculate difference in years
years_diff = invoice_year - cohort_year

# Calculate difference in months
months_diff = invoice_month - cohort_month

# Calculate difference in days
days_diff = invoice_day - cohort_day

# Extract the difference in days from all previous values
online['CohortIndex'] = years_diff * 365 + months_diff * 30 + days_diff + 1
print(online.head())

```
### Calculate retention rate from scratch
You have seen how to create retention and average quantity metrics table for the monthly acquisition cohorts. Now it's you time to build the retention metrics by yourself.

The online dataset has been loaded to you with monthly cohorts and cohort index assigned from this lesson. Feel free to print it in the Console.

Also, we have created a loaded a groupby object as grouping DataFrame with this command: grouping = online.groupby(['CohortMonth', 'CohortIndex'])

```python
# Count the number of unique values per customer ID
cohort_data = grouping['CustomerID'].apply(pd.Series.nunique).reset_index()

# Create a pivot 
cohort_counts = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')

# Select the first column and store it to cohort_sizes
cohort_sizes = cohort_counts.iloc[:,0]

# Divide the cohort count by cohort sizes along the rows
retention = cohort_counts.divide(cohort_sizes, axis=0)

```
### Calculate average price
You will now calculate the average price metric and analyze if there are any differences in shopping patterns across time and across cohorts.

The online dataset has been loaded to you with monthly cohorts and cohort index assigned from this lesson. Feel free to print it to the Console

```python
# Create a groupby object and pass the monthly cohort and cohort index as a list
grouping = online.groupby(['CohortMonth', 'CohortIndex'])

# Calculate the average of the unit price column
cohort_data = grouping['UnitPrice'].mean()

# Reset the index of cohort_data
cohort_data = cohort_data.reset_index()

# Create a pivot 
average_price = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='UnitPrice')
print(average_price.round(1))

```
### Visualize average quantity metric
You are now going to visualize average quantity values in a heatmap.

We have loaded pandas package as pd, and the average quantity values DataFrame as average_quantity.

Please use the console to explore it.



```python
# Import seaborn package as sns
import seaborn as sns

# Initialize an 8 by 6 inches plot figure
plt.figure(figsize=(8,6))

# Add a title
plt.title('Average Spend by Monthly Cohorts')

# Create the heatmap
sns.heatmap(data=average_quantity, annot=True, cmap='Blues')
plt.show()

```
### Calculate Spend quartiles (q=4)
We have created a dataset for you with random CustomerID and Spend values as data. You will now use this dataset to group customers into quartiles based on Spend values and assign labels to each of them.

pandas library as been loaded as pd. Feel free to print the data to the console.


```python
CustomerID  Spend
0           0    137
1           1    335
2           2    172
3           3    355
4           4    303
5           5    233
6           6    244
7           7    229

```
```python
# Create a spend quartile with 4 groups - a range between 1 and 5
spend_quartile = pd.qcut(data['Spend'], q=4, labels=range(1,5))

# Assign the quartile values to the Spend_Quartile column in data
data['Spend_Quartile'] = spend_quartile

# Print data with sorted Spend values
print(data.sort_values('Spend'))

     CustomerID  Spend Spend_Quartile
    0           0    137              1
    2           2    172              1
    7           7    229              2
    5           5    233              2
    6           6    244              3
    4           4    303              3
    1           1    335              4
    3           3    355              4

```
### Calculate Recency deciles (q=10)
We have created a dataset for you with random CustomerID and Recency_Days values as data. You will now use this dataset to group customers into quartiles based on Recency_Days values and assign labels to each of them.

Be cautious about the labels for this exercise. You will see that the labels are inverse, and will required one additional step in separately creating them. If you need to refresh your memory on the process of creating the labels, check out the slides!

The pandas library as been loaded as pd. Feel free to print the data to the console.
```python
# Store labels from 4 to 1 in a decreasing order
r_labels = list(range(4, 0, -1))

# Create a spend quartile with 4 groups and pass the previously created labels 
recency_quartiles = pd.qcut(data['Recency_Days'], q=4, labels=r_labels)

# Assign the quartile values to the Recency_Quartile column in `data`
data['Recency_Quartile'] = recency_quartiles 

# Print `data` with sorted Recency_Days values
print(data.sort_values('Recency_Days'))

    CustomerID  Recency_Days Recency_Quartile
    0           0            37                4
    3           3            72                4
    7           7           133                3
    6           6           203                3
    1           1           235                2
    4           4           255                2
    5           5           393                1
    2           2           396                1



```
### Calculate RFM values
Calculate Recency, Frequency and Monetary values for the online dataset we have used before - it has been loaded for you with recent 12 months of data. There's a TotalSum column in the online dataset which has been calculated by multiplying Quantity and UnitPrice: online['Quantity'] * online['UnitPrice'].

Also, we have created a snapshot_date variable that you can use to calculate recency. Feel free to print the online dataset and the snapshot_date into the Console. The pandas library is loaded as pd, and datetime as dt.
```python
# Calculate Recency, Frequency and Monetary value for each customer 
datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})

# Rename the columns 
datamart.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'TotalSum': 'MonetaryValue'}, inplace=True)

# Print top 5 rows
print(datamart.head())

            Recency  Frequency  MonetaryValue
    CustomerID                                   
    12747             3         25         948.70
    12748             1        888        7046.16
    12749             4         37         813.45
    12820             4         17         268.02
    12822            71          9         146.15


```

```python


```

```python


```
```python


```